# Questions for Yevgen

- [ ] Why do we divide the `KLP` with `neff`
- [ ] You take the mean of `RL + beta*KLZ` in the full loss. Why? And doesn't that mean it should be changed in the `train.py`  
- [ ] ask about the variance - why is it 2
- [ ] why is the logvar constant - why is it -5


# Notes meeting 15/04
Loss has three components
Lots of weights we have to scale down, make the weights as much normal as possible 
(second option: number of the full dataset)

Neff, should around 2thousand and something

Another function for evaluate the logP - calculate the probability (not looking at the parameters?) using a modified loss (we don’t for back.. but see the correlation) (we get the diff back)….if we do basin we need to do it reassemble, cause we need to reduce the variance 


`mt_elbos` # mutant elbos
It return the expected values

**Third picture** we evaluate the network and has to be implemented in the train.py

**Fourth picture** is taken form the init function 

**Fifth picture**
- (up left) sum of reconstruction loss
- (up right) useful to check if code is correct 
- (correlation) took not all the epoch but some of them (huge calculations) in the end it converges and reach the result mentioned in the paper

Questions:
- Last epoch might be have a lower number of samples (related in the picture with the error evaluation KLP)
- variance: something in the article? or trick to make it work (init values away from 0)
- C: is for compression (we have lot of information in W, and narrow it down)
- make_variational_linear(): we want our own control of the weights, we register two parameters - this is our prior for the weights, we sample from the distribution before the propagation (we have to call it once….?)
- why we sum only the last layer/dimension `.sum(-1)`, KLP is a vector ?????? Didn’t understand what he said tbh

More stuff to include later (a little bit…):
- batch normalisation
- narrow down with the C

Regarding the presentation/exam:
- Know sparsity??? Turn off some of the positions?
- Bayesian, to regularise and make it more general 

In [ ]:
def loss(self, recon_x, x, mu, logvar):
    # RL, Reconstruction loss
    # RL = F.nll_loss(recon_x, x.argmax(dim=1), reduction='none).sum(-1)
    RL = (-recon_x*x).sum(-2).sum(-1)
    # RL2 = F.nll_loss(recon_x, x.argmax(dim=1), reduction='none').sum(-1)

    # KL, Kullback-Leibler divergence loss (for `z`):
    # KLZ = -0.5 * (1 + logvar - mu.pow(2) - logvar.exp()).sum(-1)
    KLZ_q = Normal(mu, logvar.mul(1/2).exp())
    KLZ_p = Normal(torch.zeros_like(mu), torch.ones_like(logvar))
    KLZ   = kl_divergence(KLZ_q, KLZ_p).sum(-1)

    # KLP, Kullback-Leibler divergence loss (for network parameters):
    KLP = 0
    for l in self.variational_layers:
        weight_mu, weight_std = l.weight_mean, l.weight_logvar.mul(1/2).exp()
        q_weight = Normal(weight_mu, weight_std)

        if l.name == 'S':
            p_weight = Normal(torch.zeros_like(weight_mu) - 9.305, torch.zeros_like(weight_std) + 4)
        else:
            p_weight = Normal(torch.zeros_like(weight_mu), torch.ones_like(weight_std))
        
        KLP += kl_divergence(q_weight, p_weight).sum()

        if l.bias is None: continue

        bias_mu, bias_std = l.bias_mean, l.bias_logvar.mul(1/2).exp()
        q_bias = Normal(bias_mu, bias_std)
        p_bias = Normal(torch.zeros_like(bias_mu), torch.ones_like(bias_std))
        KLP += kl_divergence(q_bias, p_bias).sum()


    # Variational parameters: lambda (lamb) and w_out_b
    lamb_mu, lamb_std = self.lamb_mean, self.lamb_logvar.mul(1/2).exp()
    q_lamb = Normal(lamb_mu, lamb_std)
    p_lamb = Normal(torch.zeros_like(lamb_mu), torch.ones_like(lamd_std))
    KLP   += kl_divergence(q_lamb, p_lamb).sum()

    W_out_b_mu, W_out_b_std = self.W_out_b_mean, self.W_out_b_logvar.mul(1/2).exp()
    q_W_out_b = Normal(W_out_b_mu, W_out_b_std)
    p_W_out_b = Normal(torch.zeros_like(W_out_b_mu), torch.ones_like(W_out_b_std))
    KLP      += kl_divergence(q_W_out_b, p_W_out_b).sum()

    KLP /= self.neff
    loss = (RL + self.beta * KLZ).mean() + KLP

    KLP = torch.tensor([0], requires_grad=False) if KLP == 0 else KLP
    return loss, RL.mean(), KLZ.mean(), KLP


In [ ]:
def logp(self, batch, rand = False):
    ''' Returns individual log likelihoods of the batch'''

    mu, logvar = self.encode(batch)
    z          = self.reparameterize(mu, logvar) if rand else mu

    recon      = self.decode(z)
    recon      = recon.view(-1, self.alphabet_len, self.seq_len)
    recon      = recon.log_softmax(dim=1)

    # logp     = F.nll_loss(recon, batch.argmax(dim=1), reduction='none').sum(-1)
    logp       = (-recon*batch).sum(-2).sum(-1)
    kl         = -0.5 * (1 + logvar - mu.pow(2) - logvar.exp()).sum(-1)
    elbo       = logp + kl

    return elbo

In [ ]:
def get_cor_ensamble(batch, mutants_values, model, ensambles = 512, rand = True):
    model.eval()

    mt_elbos, wt_elbos = 0, 0

    for i in range(ensambles):
        if i and (i % 2 == 0):
            print(f"\tReached {i}/rand={rand}", " "*32, end="\r")

        elbos     = model.logp(batch, rand=rand).detach().cpu()
        wt_elbos += elbos[0]
        mt_elbos += elbos[1:]

    print()

    diffs  = (mt_elbos / ensambles) - (wt_elbos / ensambles)
    cor, _ = spearmanr(mutants_values, diffs)
    
    return cor

----
**Code sent on zoom chat**

In [ ]:
# Loss() in vae.py → 

from torch.distributions import kl_divergence
KLZ_q = Normal(mu, logvar.mul(1/2).exp())
KLZ_p = Normal(torch.zeros_like(mu), torch.ones_like(logvar))
KLZ   = kl_divergence(KLZ_q, KLZ_p).sum(-1)


In [ ]:
# Loss of recostration - distance network results and expected
# we have categorical distribution (for every distribution??)
# every column is one position

from torch.nn import functional as F
F.nll_loss(recon_x, x.argmax(dim=1), reduction='none').sum(-1)

# we don’t do any reduction - otherwise mess up with the ….tensor?

**super messy notes**

Loss() in vae.py → 

```
from torch.distributions import kl_divergence
KLZ_q = Normal(mu, logvar.mul(1/2).exp())
KLZ_p = Normal(torch.zeros_like(mu), torch.ones_like(logvar))
KLZ   = kl_divergence(KLZ_q, KLZ_p).sum(-1)
```


Loss of recustring - distance network results and expected
we have categorical distribution (for every distribution??)
every column is one position

```
from torch.nn import functional as F
F.nll_loss(recon_x, x.argmax(dim=1), reduction='none').sum(-1)
```

we don’t do any reduction - otherwise mess up with the ….tensor?

...if we do in log...


→ Sparsity? → group sparsity prior: L1 Penalty? → MATRIX!! Hidden unity fully connected
Turn connection, does not affect… [somthing] multiply on top on hidden layer


WE WANT THE FUNKY PART 

[..]

look at similarities, calculate similarities, reduce their weight on the train

created a layer [look at code shared on slack]
lambda has baias - this is not probabilistic 

[ picture2 ] in the constructor of VAE --- we have to do some tricks (for?)

[picture3 it also includes the lambda] multiply matrices ….and?extract matrix from the layer, do manipulations, → it’s the calculation of the 3 matrices → the matrix that we learn (...is going to be sparse,... turn on the weight)

PRINT OUT THE TENSOR → FIGURE OUT THE SHAPE OF STUFF (this step to learn and understand)

[picture 4] forward inside the encoder method - it returns mean, …., latent space

STEFAN QUESTION: Right before the last layer (in the encoder)

[Eugenio didn’t do the ensembling]

SequenceRate
calculate huge matrix with similarities (hamming distance)
look at how similar is protein, a comparison between them
we can calculate this in batches (this will speed up the creation of the matrix)

(new picture)we’ll get a list of weights ---> and retrain???? not sure about this..
it fully deterministic
[what we have to do then?]
HINT: data loader, combined with a sampler (look at new picture)

```
from torch.utils.data import TensorDataset, DataLoader, WeightedRandomSampler
.backward(<<< Parameter >>>) // used for scale
```


“Second, we encourage correlation between amino acid usage, by convolving the final layer with a width-1 convolution operation.” → this is the dictionary → the DEMATRIX?


SCALE UP THE NETWORK (for sparsity) with more hidden layer

linear layer is a fully connected layer ;)


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=35838f82-2ce6-4453-9bd2-2d87a43af151' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>